In [1]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

### Test Data
We will be using the Ordnance Survey Open Roads dataset converted into a graph.  
A sample is shown below:

![Road_Graph](./img/leith3.png)

This is a weighted graph, using the edge distance in metres.  
__Note__: The Open Roads dataset starts with vertex ID 1 which the cuGraph analytics assume a zero-based starting ID.  
*Contains OS data © Crown copyright and database right (2019)*  
Licensed under [Open Government Licence (OGL) V3](http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/)

### Read the source data and adjust the vertex IDs

In [2]:
# Test file  - Read OS Open Roads as a graph. (Store dataset in ./data folder)
datafile='./data/gb_road_graph_20190523.csv'

In [3]:
# Read the data file with lenght of edge in metres as weight
cols = ["src", "dst","length"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32"),
        ("length", "float64"),
        ])

gdf = cudf.read_csv(datafile, names=cols, dtype=list(dtypes.values()) ,skiprows=1)

In [4]:
gdf

<cudf.DataFrame ncols=3 nrows=7347806 >

In [5]:
# Display the graph adjacency matrix
print(gdf)

    src  dst  length
0   16   20   162.0
1   20   16   162.0
2   56   55   855.0
3   55   56   855.0
4   72   22   382.0
5   22   72   382.0
6   72   71   270.0
7   71   72   270.0
8   17   18    48.0
9   18   17    48.0
[7347796 more rows]


In [6]:
# Need to shift the vertex IDs to start with zero rather than one (next version of cuGraph will fix this issue)
gdf["src"] = gdf["src"] - 1
gdf["dst"] = gdf["dst"] - 1

In [7]:
gdf

<cudf.DataFrame ncols=3 nrows=7347806 >

In [8]:
# Display the graph adjacency matrix
print(gdf)

    src  dst  length
0   15   19   162.0
1   19   15   162.0
2   55   54   855.0
3   54   55   855.0
4   71   21   382.0
5   21   71   382.0
6   71   70   270.0
7   70   71   270.0
8   16   17    48.0
9   17   16    48.0
[7347796 more rows]


### Create the Road Graph and call SSSP

In [9]:
%%time
# create a Graph 
G = cugraph.Graph()
G.add_edge_list(gdf["src"], gdf["dst"],gdf['length'])

CPU times: user 421 µs, sys: 0 ns, total: 421 µs
Wall time: 428 µs


In [10]:
%%time
# Call cugraph.sssp to get the distances from vertex 302535 (Leith Town Hall & Theatre):
df = cugraph.sssp(G,302535)

CPU times: user 1.98 s, sys: 819 ms, total: 2.8 s
Wall time: 2.79 s


### Calculate 500 miles walking distance

In [11]:
# Calculate 500 miles walking distance (804672 metres)
print(df.query("distance==804672.00"))

    vertex  distance
2217019  2217019  804672.0
2225573  2225573  804672.0


___
Copyright (c) 2019, NVIDIA CORPORATION.

Adapted by John Murray

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___